In [11]:
from controllers import *
from cyipopt import minimize_ipopt
# k_1,k_2,k_3 = 1500898.8475089157, 51911.41903311291, 4.166999999999997
# #E_1,E_2,E_3 = 7.94305039e+04,7.96897538e+04,1.24494237e+05
# Cp1,Cp2 = 1.8097737040262707, 2.6356562630366525
Coeff1 = F1/V1
Coeff2 = F1/V2
Coeff3 = F2_new/V2
@njit
def model_makestep(x,u, steps=100, hc=1e-4):
    C_B1,C_DEB1,C_E1,C_EB1,T_1,C_B2,C_DEB2,C_E2,C_EB2,T_2 = x
    Q1,Q2 = u
    for _ in range(steps):
        r1_1  = k_1*math.exp(-E_1/R/(T_1+T_1s))*((C_B1+C_B1s)*(C_E1+C_E1s))
        r1s_1 = k_1*math.exp(-E_1/R/T_1s)*(C_B1s*C_E1s)
        r2_1  = k_2*math.exp(-E_2/R/(T_1+T_1s))*((C_EB1+C_EB1s)*(C_E1+C_E1s))
        r2s_1 = k_2*math.exp(-E_2/R/T_1s)*(C_EB1s*C_E1s)
        r3_1  = k_3*math.exp(-E_3/R/(T_1+T_1s))*((C_B1+C_B1s)*(C_DEB1+C_DEB1s))
        r3s_1 = k_3*math.exp(-E_3/R/T_1s)*(C_B1s*C_DEB1s) 

        r1_2  = k_1*math.exp(-E_1/R/(T_2+T_2s))*((C_B2+C_B2s)*(C_E2+C_E2s))
        r1s_2 = k_1*math.exp(-E_1/R/T_2s)*(C_B2s*C_E2s)
        r2_2  = k_2*math.exp(-E_2/R/(T_2+T_2s))*((C_EB2+C_EB2s)*(C_E2+C_E2s))
        r2s_2 = k_2*math.exp(-E_2/R/T_2s)*(C_EB2s*C_E2s)
        r3_2  = k_3*math.exp(-E_3/R/(T_2+T_2s))*((C_B2+C_B2s)*(C_DEB2+C_DEB2s))
        r3s_2 = k_3*math.exp(-E_3/R/T_2s)*(C_B2s*C_DEB2s)

        C_B1   = max(-C_B1s,C_B1+hc*(Coeff1*(-C_B1)                            - r1_1 + r1s_1  - r3_1 + r3s_1))
        C_B2   = max(-C_B2s,C_B2+hc*(Coeff2*(C_B1-C_B2)     + Coeff3*(-C_B2)   - r1_2 + r1s_2  - r3_2 + r3s_2))
        C_DEB1 = max(-C_DEB1s,C_DEB1+hc*(Coeff1*(-C_DEB1)                          + r2_1 - r2s_1 - r3_1 + r3s_1))
        C_DEB2 = max(-C_DEB2s,C_DEB2+hc*(Coeff2*(C_DEB1-C_DEB2) + Coeff3*(-C_DEB2) + r2_2 - r2s_2 - r3_2 + r3s_2))
        C_E1   = max(-C_E1s,C_E1+hc*(Coeff1*(-C_E1)                            - r1_1 + r1s_1  - r2_1 + r2s_1))
        C_E2   = max(-C_E2s,C_E2+hc*(Coeff2*(C_E1-C_E2)     + Coeff3*(-C_E2)   - r1_2 + r1s_2  - r2_2 + r2s_2))
        C_EB1  = max(-C_EB1s,C_EB1+hc*(Coeff1*(-C_EB1)                           + r1_1 - r1s_1  - r2_1 + r2s_1 + 2*(r3_1-r3s_1)))
        C_EB2  = max(-C_EB2s,C_EB2+hc*(Coeff2*(C_EB1-C_EB2)   + Coeff3*(-C_EB2)  + r1_2 - r1s_2  - r2_2 + r2s_2 + 2*(r3_2-r3s_2)))
        T_1    = max(-T_1s,T_1+hc*(Coeff1*(-T_1)                    - H_1*(r1_1-r1s_1)/(rhoe_L1*Cp1) - H_2*(r2_1-r2s_1)/(rhoe_L1*Cp1)- H_3*(r3_1-r3s_1)/(rhoe_L1*Cp1) +Q1/(rhoe_L1 * Cp1 * V1)))
        T_2    = max(-T_2s,T_2+hc*(Coeff2*(T_1-T_2) + Coeff3*(-T_2) - H_1*(r1_2-r1s_2)/(rhoe_L2*Cp2) - H_2*(r2_2-r2s_2)/(rhoe_L2*Cp2)- H_3*(r3_2-r3s_2)/(rhoe_L2*Cp2) +Q2/(rhoe_L2 * Cp2 * V2)))
    return np.array([C_B1,C_DEB1,C_E1,C_EB1,T_1,C_B2,C_DEB2,C_E2,C_EB2,T_2])
def eval_f(u):#L=xQx+uRu
    offset = 0
    x = stateRecord[:,:,main_iteration].reshape(-1)
    
    # Assuming the state vector includes all species and temperature for both reactors
    

    # Numerical Integration
    for controlIteration in range(controlTimeStepsPerMPC):
        Q1    = u.item(NUM_U*controlIteration + 0)  # Heat input for reactor 1
        Q2    = u.item(NUM_U*controlIteration + 1)  # Heat input for reactor 2
        
        # Update states for both reactors
        x = model_makestep(x, [Q1,Q2], controlStepSize, hcMPC)
        # if np.isnan(x).any() or np.isinf(x).any():
        #     return 1e6
        C_EB2 = x.item(8)
        # Cost calculation:
        # - Minimize consumption of Benzene (B)
        # - Maximize production of Ethylbenzene (EB)
        # - Minimize heat usage (Q^2 for both reactors)
        
        # Concentration of EthylBenzene (B) in last reactor
        finalEthylBenzene = C_EB2
        
        # Heat usage
        heat_usage = (Q1)**2 + (Q2)**2
        
         # Cost function components:
        # - Maximize EB production (negative sign because we're minimizing, but we want to maximize this)
        # - Minimize heat usage (positive sign to minimize)
        offset += -A1[main_iteration]*finalEthylBenzene #+ A2[main_iteration]*heat_usage
    #print(offset*delta)
    return offset * delta
def objective(params):
    vdot_components = Vdot(params)
    diff = sum(differenceCalc(vdot_components))
    return diff
#Example initial guess for inputs
initial_params = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])  # Example values
#Constraints can be added if needed, e.g., for non-negative inputs
constraints = ({'type': 'ineq', 'fun': lambda x: x})  # All inputs must be >= 0
#Optimization
result = minimize_ipopt(
    objective, 
    initial_params, 
    #method='SLSQP',  # Sequential Least SQuares Programming for handling constraints if any
    #method='Nelder-Mead',
    constraints=constraints,
    options={'disp': True, 'maxiter': 1000, 'tol':1e-12}
)
np.set_printoptions(precision=60, suppress=True)
print(f"Optimized Parameters for P matrix:{result.x}")
print("Final Disparity Value:", result.fun)

#Check results
optimized_vdot_components = Vdot(result.x)
print(f"Optimized Vdot Components:{optimized_vdot_components}")
print("Mean of Optimized Vdot Components:", np.mean(optimized_vdot_components))
print("CV of Optimized Vdot Components:", np.std(optimized_vdot_components)/np.mean(optimized_vdot_components))

np.float64(0.0008299190228481955)

In [ ]:
exec("\n".join(f"{m}" for m in open('packages.txt').read().splitlines() if m.strip()))
# Numpy adds some useful numerical types and functions
# Cantera will handle thermodynamic properties
# Pint gives us some helpful unit conversion
ureg = UnitRegistry()
Q_ = ureg.Quantity # We will use this to construct quantities (value + unit)

#Steady State Values of state variables
Q1s     = Q_(-1074.63 ,'kJ/s').magnitude
Q2s     = Q_(-6768.83 ,'kJ/s').magnitude
C_B1s   = Q_(5.551990403223146 ,'kmol/m^3').magnitude
C_DEB1s = Q_(8.75999437550174e-006 ,'kmol/m^3').magnitude
C_E1s   = Q_(4.333830798637546 ,'kmol/m^3').magnitude
C_EB1s  = Q_(0.539305 ,'kmol/m^3').magnitude
T_1s    = Q_(321.154 ,'K').magnitude
C_B2s   = Q_(1.309395934989596 ,'kmol/m^3').magnitude
C_DEB2s = Q_(0.007767870421433004 ,'kmol/m^3').magnitude
C_E2s   = Q_(0.1962010989664658 ,'kmol/m^3').magnitude
C_EB2s  = Q_(4.211820652459363 ,'kmol/m^3').magnitude
T_2s    = Q_(442.9880212435662 ,'K').magnitude

#Steady State Values of state variables
C_B10s   = Q_(5.53823 ,'kmol/m^3').magnitude
C_DEB10s = Q_(0.0 ,'kmol/m^3').magnitude
C_E10s   = Q_(4.43068 ,'kmol/m^3').magnitude
C_EB10s  = Q_(0.0 ,'kmol/m^3').magnitude
T_10s    = Q_(350 ,'K').magnitude
C_B20s   = Q_(5.01953 ,'kmol/m^3').magnitude
C_DEB20s = Q_(0.0 ,'kmol/m^3').magnitude
C_E20s   = Q_(4.0162 ,'kmol/m^3').magnitude
C_EB20s  = Q_(0.0 ,'kmol/m^3').magnitude
T_20s    = Q_(350 ,'K').magnitude

#Non-linear System Parameters
rhoe_L1  = Q_(640.695,'kg/m^3').magnitude
rhoe_L2  = Q_(608.734,'kg/m^3').magnitude

#Volume and flow rates
F1     = (Q_(443.484,'kmol/hr')*Q_(97.7634,'cc/mol')).to('m^3/s').magnitude
F2_net = (Q_(549.63,'kmol/hr')*Q_(157.801,'cc/mol')).to('m^3/s').magnitude
F2_new = F2_net - F1
V1 = Q_(60.0,'m^3').magnitude
V2 = Q_(60.0,'m^3').magnitude

#Thermodynamic Parameters
E_1 = Q_(16.9963,'kcal/mol').to('kJ/kmol').magnitude
E_2 = Q_(19.9866,'kcal/mol').to('kJ/kmol').magnitude
E_3 = Q_(14.99,'kcal/mol').to('kJ/kmol').magnitude
H_1 = Q_(-114.13,'kJ/mol').to('kJ/kmol').magnitude
H_2 = Q_(-112.56,'kJ/mol').to('kJ/kmol').magnitude
H_3 = Q_(-1.57,'kJ/mol').to('kJ/kmol').magnitude
R   = Q_(8.31446261815324,'kJ/kmol/K').magnitude
# https://www.sciencedirect.com/science/article/pii/S1876619615000868
# B + E -> EB -114.13 kJ/mole
# EB + E -> DEB -112.56 kJ/mole
# DEB + B -> 2EB -1.57 kJ/mole

# Initial Guesses
Cp1 = Q_(0.238846,'cal/g/K').to('kJ/kg/K').magnitude
Cp2 = Q_(0.238846,'cal/g/K').to('kJ/kg/K').magnitude
k_1 = Q_(1.528*10**6,'m^3/kmol/s').magnitude
k_2 = Q_(27780,'m^3/kmol/s').magnitude
k_3 = Q_(0.4167,'m^3/kmol/s').magnitude
k=np.array([k_1,k_2,k_3])
E=np.array([E_1,E_2,E_3])
rxnCoeffs=np.array([[1,0,1,0],[0,0,1,1],[1,1,0,0]])
dH=np.array([H_1,H_2,H_3])


#Use .magnitude instead of
dC_dt = lambda F, V, C0, C, T, k, E, R, index, rxns,rxnCoeffs: (
    # Net flow in and out of the reactor for species at index
    (np.sum(F * C0[index]) - np.sum(F) * C[index]) / V 
    # Reaction rates contribution
    + np.sum(rxns * k * np.exp(-E / (R * T)) * np.sum(rxnCoeffs*C,1))
)
def seriesCSTR(params):
    k_1,k_2,k_3,Cp1,Cp2 = params  # Unpack the parameters
    dCb1_dt   = F1*(C_B10s-C_B1s)/V1     - k_1*exp(-E_1/R/T_1s)*(C_B1s*C_E1s)  - k_3*exp(-E_3/R/T_1s)*(C_B1s*C_DEB1s)
    dCb2_dt   = F1*(C_B1s-C_B2s)/V2      + F2_new*(C_B20s-C_B2s)/V2            - k_1*exp(-E_1/R/T_2s)*(C_B2s*C_E2s)  - k_3*exp(-E_3/R/T_2s)*(C_B2s*C_DEB2s)
    dCdeb1_dt = F1*(C_DEB10s-C_DEB1s)/V1 + k_2*exp(-E_2/R/T_1s)*(C_EB1s*C_E1s) - k_3*exp(-E_3/R/T_1s)*(C_B1s*C_DEB1s)
    dCdeb2_dt = F1*(C_DEB1s-C_DEB2s)/V2  + F2_new*(C_DEB20s-C_DEB2s)/V2        + k_2*exp(-E_2/R/T_2s)*(C_EB2s*C_E2s) - k_3*exp(-E_3/R/T_2s)*(C_B2s*C_DEB2s)
    dCe1_dt   = F1*(C_E10s-C_E1s)/V1     - k_1*exp(-E_1/R/T_1s)*(C_B1s*C_E1s)  - k_2*exp(-E_2/R/T_1s)*(C_EB1s*C_E1s)
    dCe2_dt   = F1*(C_E1s-C_E2s)/V2      + F2_new*(C_E20s-C_E2s)/V2            - k_1*exp(-E_1/R/T_2s)*(C_B2s*C_E2s)  - k_2*exp(-E_2/R/T_2s)*(C_EB2s*C_E2s)
    dCeb1_dt  = F1*(C_EB10s-C_EB1s)/V1   + k_1*exp(-E_1/R/T_1s)*(C_B1s*C_E1s)  - k_2*exp(-E_2/R/T_1s)*(C_EB1s*C_E1s) + 2*k_3*exp(-E_3/R/T_1s)*(C_B1s*C_DEB1s)
    dCeb2_dt  = F1*(C_EB1s-C_EB2s)/V2    + F2_new*(C_EB20s-C_EB2s)/V2          + k_1*exp(-E_1/R/T_2s)*(C_B2s*C_E2s)  - k_2*exp(-E_2/R/T_2s)*(C_EB2s*C_E2s) + 2*k_3*exp(-E_3/R/T_2s)*(C_B2s*C_DEB2s)
    dT1_dt    = F1*(T_10s-T_1s)/V1                         - H_1*k_1*exp(-E_1/R/T_1s)*(C_B1s*C_E1s)/(rhoe_L1*Cp1) - H_2*k_2*exp(-E_2/R/T_1s)*(C_EB1s*C_E1s)/(rhoe_L1*Cp1)- H_3*k_3*exp(-E_3/R/T_1s)*(C_B1s*C_DEB1s)/(rhoe_L1*Cp1) +Q1s/(rhoe_L1 * Cp1 * V1)
    dT2_dt    = F1*(T_1s-T_2s)/V2 + F2_new*(T_20s-T_2s)/V2 - H_1*k_1*exp(-E_1/R/T_2s)*(C_B2s*C_E2s)/(rhoe_L2*Cp2) - H_2*k_2*exp(-E_2/R/T_2s)*(C_EB2s*C_E2s)/(rhoe_L2*Cp2)- H_3*k_3*exp(-E_3/R/T_2s)*(C_B2s*C_DEB2s)/(rhoe_L2*Cp2) +Q2s/(rhoe_L2 * Cp2 * V2)
    return np.array([dCb1_dt,dCdeb1_dt,dCe1_dt,dCeb1_dt,dT1_dt,dCb2_dt,dCdeb2_dt,dCe2_dt,dCeb2_dt,dT2_dt], dtype=float)
print(seriesCSTR(np.array([k_1,k_2,k_3,Cp1,Cp2], dtype=float)))
test = least_squares(seriesCSTR,np.array([k_1,k_2,k_3,Cp1,Cp2], dtype=float),gtol=2.223e-16,ftol=2.223e-16,xtol=2.223e-16,bounds = ([0.1*k_1,0.1*k_2,0.1*k_3,0.1*Cp1,0.1*Cp2],[k_1*10,k_2*10,k_3*10,Cp1*10,Cp2*10]))
#print(f"The rate of change of B is {dCb_dt}.\nThe rate of change of DEB is {dCdeb_dt}.\nThe rate of change of E is {dCe_dt}.\nThe rate of change of EB is {dCeb_dt}.\nThe rate of change of T is {dT1_dt}.")
np.printoptions(precision=3, suppress=False)
print(f"The effecite steady-state rate of change for each species is {Q_(test.fun[0:4],'mol/m^3/s')*Q_(60,'m^3'):.2e} for CSTR1")
print(f"The effecite steady-state rate of change for Temperature is {Q_(test.fun[4],'K/s'):.2e} for CSTR1")
print(f"The effecite steady-state rate of change for each species is {Q_(test.fun[5:9],'mol/m^3/s')*Q_(60,'m^3'):.2e} for CSTR2")
print(f"The effecite steady-state rate of change for Temperature is {Q_(test.fun[9],'K/s'):.2e} for CSTR2")

print(f"The solved values for k are {Q_(test.x[0:3],'m^3/kmol/s'):.2e}.\nThe solved values for Cp are {Q_(test.x[3:],'kJ/kg/K'):.2e}.")

k_1 = Q_(1.528*10**6,'m^3/kmol/s').magnitude
k_2 = Q_(27780,'m^3/kmol/s').magnitude
k_3 = Q_(0.4167,'m^3/kmol/s').magnitude
k=Q_([k_1,k_2,k_3],'m^3/kmol/s')
Cp1 = Q_(0.238846,'cal/g/K').to('kJ/kg/K').magnitude
Cp2 = Q_(0.238846,'cal/g/K').to('kJ/kg/K').magnitude
Cp=Q_([Cp1,Cp2],'cal/g/K').to('kJ/kg/K')
print(f"The original values for k are {k:.2e}.\nThe original values for Cp are {Cp:.2e}.")
print(test)

In [ ]:
exec("\n".join(f"{m}" for m in open('packages.txt').read().splitlines() if m.strip()))
# Numpy adds some useful numerical types and functions
# Cantera will handle thermodynamic properties
# Pint gives us some helpful unit conversion
ureg = UnitRegistry()
Q_ = ureg.Quantity # We will use this to construct quantities (value + unit)

#Steady State Values of state variables
Q1s     = Q_(-1074.63 ,'kJ/s')
Q2s     = Q_(-6768.83 ,'kJ/s')
C_B1s   = Q_(5.551990403223146 ,'kmol/m^3')
C_DEB1s = Q_(8.75999437550174e-006 ,'kmol/m^3')
C_E1s   = Q_(4.333830798637546 ,'kmol/m^3')
C_EB1s  = Q_(0.539305 ,'kmol/m^3')
T_1s    = Q_(321.154 ,'K')
C_B2s   = Q_(1.309395934989596 ,'kmol/m^3')
C_DEB2s = Q_(0.007767870421433004 ,'kmol/m^3')
C_E2s   = Q_(0.1962010989664658 ,'kmol/m^3')
C_EB2s  = Q_(4.211820652459363 ,'kmol/m^3')
T_2s    = Q_(442.9880212435662 ,'K')

#Steady State Values of state variables
C_B10s   = Q_(5.53823 ,'kmol/m^3')
C_DEB10s = Q_(0.0 ,'kmol/m^3')
C_E10s   = Q_(4.43068 ,'kmol/m^3')
C_EB10s  = Q_(0.0 ,'kmol/m^3')
T_10s    = Q_(350 ,'K')
C_B20s   = Q_(5.01953 ,'kmol/m^3')
C_DEB20s = Q_(0.0 ,'kmol/m^3')
C_E20s   = Q_(4.0162 ,'kmol/m^3')
C_EB20s  = Q_(0.0 ,'kmol/m^3')
T_20s    = Q_(350 ,'K')

#Non-linear System Parameters
rhoe_L1  = Q_(640.695,'kg/m^3')
rhoe_L2  = Q_(608.734,'kg/m^3')

#Volume and flow rates
F1     = (Q_(443.484,'kmol/hr')*Q_(97.7634,'cc/mol')).to('m^3/s')
F2_net = (Q_(549.63,'kmol/hr')*Q_(157.801,'cc/mol')).to('m^3/s')
F2_new = F2_net - F1
V1 = Q_(60.0,'m^3')
V2 = Q_(60.0,'m^3')

#Thermodynamic Parameters
E_1 = Q_(16.9963,'kcal/mol').to('kJ/kmol')
E_2 = Q_(19.9866,'kcal/mol').to('kJ/kmol')
E_3 = Q_(14.99,'kcal/mol').to('kJ/kmol')
H_1 = Q_(-114.13,'kJ/mol').to('kJ/kmol')
H_2 = Q_(-112.56,'kJ/mol').to('kJ/kmol')
H_3 = Q_(-1.57,'kJ/mol').to('kJ/kmol')
R   = Q_(8.31446261815324,'kJ/kmol/K')
# https://www.sciencedirect.com/science/article/pii/S1876619615000868
# B + E -> EB -114.13 kJ/mole
# EB + E -> DEB -112.56 kJ/mole
# DEB + B -> 2EB -1.57 kJ/mole

# Initial Guesses
Cp1 = Q_(0.238846,'cal/g/K').to('kJ/kg/K')
Cp2 = Q_(0.238846,'cal/g/K').to('kJ/kg/K')
k_1 = Q_(1.528*10**6,'m^3/kmol/s')
k_2 = Q_(27780,'m^3/kmol/s')
k_3 = Q_(0.4167,'m^3/kmol/s')

def seriesCSTR(params):
    k_1,k_2,k_3,Cp1,Cp2 = params  # Unpack the parameters
    dCb1_dt   = F1*(C_B10s-C_B1s)/V1     - k_1*exp(-E_1/R/T_1s)*(C_B1s*C_E1s)  - k_3*exp(-E_3/R/T_1s)*(C_B1s*C_DEB1s)
    dCb2_dt   = F1*(C_B1s-C_B2s)/V2      + F2_new*(C_B20s-C_B2s)/V2            - k_1*exp(-E_1/R/T_2s)*(C_B2s*C_E2s)  - k_3*exp(-E_3/R/T_2s)*(C_B2s*C_DEB2s)
    dCdeb1_dt = F1*(C_DEB10s-C_DEB1s)/V1 + k_2*exp(-E_2/R/T_1s)*(C_EB1s*C_E1s) - k_3*exp(-E_3/R/T_1s)*(C_B1s*C_DEB1s)
    dCdeb2_dt = F1*(C_DEB1s-C_DEB2s)/V2  + F2_new*(C_DEB20s-C_DEB2s)/V2        + k_2*exp(-E_2/R/T_2s)*(C_EB2s*C_E2s) - k_3*exp(-E_3/R/T_2s)*(C_B2s*C_DEB2s)
    dCe1_dt   = F1*(C_E10s-C_E1s)/V1     - k_1*exp(-E_1/R/T_1s)*(C_B1s*C_E1s)  - k_2*exp(-E_2/R/T_1s)*(C_EB1s*C_E1s)
    dCe2_dt   = F1*(C_E1s-C_E2s)/V2      + F2_new*(C_E20s-C_E2s)/V2            - k_1*exp(-E_1/R/T_2s)*(C_B2s*C_E2s)  - k_2*exp(-E_2/R/T_2s)*(C_EB2s*C_E2s)
    dCeb1_dt  = F1*(C_EB10s-C_EB1s)/V1   + k_1*exp(-E_1/R/T_1s)*(C_B1s*C_E1s)  - k_2*exp(-E_2/R/T_1s)*(C_EB1s*C_E1s) + 2*k_3*exp(-E_3/R/T_1s)*(C_B1s*C_DEB1s)
    dCeb2_dt  = F1*(C_EB1s-C_EB2s)/V2    + F2_new*(C_EB20s-C_EB2s)/V2          + k_1*exp(-E_1/R/T_2s)*(C_B2s*C_E2s)  - k_2*exp(-E_2/R/T_2s)*(C_EB2s*C_E2s) + 2*k_3*exp(-E_3/R/T_2s)*(C_B2s*C_DEB2s)
    dT1_dt    = F1*(T_10s-T_1s)/V1                         - H_1*k_1*exp(-E_1/R/T_1s)*(C_B1s*C_E1s)/(rhoe_L1*Cp1) - H_2*k_2*exp(-E_2/R/T_1s)*(C_EB1s*C_E1s)/(rhoe_L1*Cp1)- H_3*k_3*exp(-E_3/R/T_1s)*(C_B1s*C_DEB1s)/(rhoe_L1*Cp1) +Q1s/(rhoe_L1 * Cp1 * V1)
    dT2_dt    = F1*(T_1s-T_2s)/V2 + F2_new*(T_20s-T_2s)/V2 - H_1*k_1*exp(-E_1/R/T_2s)*(C_B2s*C_E2s)/(rhoe_L2*Cp2) - H_2*k_2*exp(-E_2/R/T_2s)*(C_EB2s*C_E2s)/(rhoe_L2*Cp2)- H_3*k_3*exp(-E_3/R/T_2s)*(C_B2s*C_DEB2s)/(rhoe_L2*Cp2) +Q2s/(rhoe_L2 * Cp2 * V2)
    return [dCb1_dt,dCdeb1_dt,dCe1_dt,dCeb1_dt,dT1_dt,dCb2_dt,dCdeb2_dt,dCe2_dt,dCeb2_dt,dT2_dt]
results = seriesCSTR([k_1,k_2,k_3,Cp1,Cp2])
for ses in results:
    print(ses)

In [ ]:
from util import *

#Use .magnitude instead of
dC_dt = lambda F, V, C0, C, T, k, E, R, index, rxns,rxnCoeffs: (
    # Net flow in and out of the reactor for species at index
    (np.sum(F * C0[index]) - np.sum(F) * C[index]) / V 
    # Reaction rates contribution
    + np.sum(rxns * k * np.exp(-E / (R * T)) * np.prod(rxnCoeffs*C,1,where=((rxnCoeffs*C)!=0)))
)
dT_dt = lambda F, V, C, T0, T, dH, rho_L, Cp, Q, k, E, R, rxnCoeffs: (
    # Net flow in and out of the reactor for species at index
    (np.sum(F * T0) - np.sum(F) * T) / V 
    # Reaction rates contribution
    + np.sum(-dH * k * np.exp(-E / (R * T))* np.prod(rxnCoeffs*C,1,where=((rxnCoeffs*C)!=0)))/(rho_L * Cp)+Q/(rho_L * Cp * V)
)

def CSTR1(params):
    k1, k2, k3, Cp1, Cp2 = params  # Unpack the parameters
    k = np.array([k1, k2, k3])

    F=np.array([F1])
    V=V1
    C0=np.array([[C_B10s],[C_DEB10s],[C_E10s],[C_EB10s]])
    C=np.array([C_B1s,C_DEB1s,C_E1s,C_EB1s])
    T=T_1s
    T0=T_10s
    rho_L=rhoe_L1
    Cp=Cp1
    Q=Q1s
    dCb_dt   = dC_dt(F, V, C0, C, T, k, E, R, 0, [-1,0,-1],rxnCoeffs)
    dCdeb_dt = dC_dt(F, V, C0, C, T, k, E, R, 1, [0,1,-1],rxnCoeffs)
    dCe_dt   = dC_dt(F, V, C0, C, T, k, E, R, 2, [-1,-1,0],rxnCoeffs)
    dCeb_dt  = dC_dt(F, V, C0, C, T, k, E, R, 3, [1,-1,2],rxnCoeffs)
    dT1_dt    = dT_dt(F, V, C, T0, T, dH, rho_L, Cp, Q, k, E, R,rxnCoeffs)
    return np.hstack([dCb_dt,dCdeb_dt,dCe_dt,dCeb_dt,dT1_dt])
def CSTR2(params):
    k1, k2, k3, Cp1, Cp2 = params  # Unpack the parameters
    k = np.array([k1, k2, k3])
    
    F=np.array([F1,F2_new])
    V=V2
    C0=np.array([[C_B1s,C_B20s],[C_DEB1s,C_DEB20s],[C_E1s,C_E20s],[C_EB1s,C_EB20s]])
    C=np.array([C_B2s,C_DEB2s,C_E2s,C_EB2s])
    T0=np.array([T_1s,T_20s])
    T=T_2s
    rho_L=rhoe_L2
    Cp=Cp2
    Q=Q2s
    dCb_dt   = dC_dt(F, V, C0, C, T, k, E, R, 0, [-1,0,-1],rxnCoeffs)
    dCdeb_dt = dC_dt(F, V, C0, C, T, k, E, R, 1, [0,1,-1],rxnCoeffs)
    dCe_dt   = dC_dt(F, V, C0, C, T, k, E, R, 2, [-1,-1,0],rxnCoeffs)
    dCeb_dt  = dC_dt(F, V, C0, C, T, k, E, R, 3, [1,-1,2],rxnCoeffs)
    dT2_dt    = dT_dt(F, V, C, T0, T, dH, rho_L, Cp, Q, k, E, R,rxnCoeffs)
    return np.hstack([dCb_dt,dCdeb_dt,dCe_dt,dCeb_dt,dT2_dt])
def seriesCSTR(params):
    return np.hstack([CSTR1(params),CSTR2(params)], dtype=float)
def seriesCSTRSS(params,*args):
    k_1,k_2,k_3,Cp1,Cp2 = params  # Unpack the parameters
    # Cp1=Cp
    # Cp2=Cp
    dCb1_dt   = F1*(C_B10s-C_B1s)/V1     - k_1*exp(-E_1/R/T_1s)*(C_B1s*C_E1s)  - k_3*exp(-E_3/R/T_1s)*(C_B1s*C_DEB1s)
    dCb2_dt   = F1*(C_B1s-C_B2s)/V2      + F2_new*(C_B20s-C_B2s)/V2            - k_1*exp(-E_1/R/T_2s)*(C_B2s*C_E2s)  - k_3*exp(-E_3/R/T_2s)*(C_B2s*C_DEB2s)
    dCdeb1_dt = F1*(C_DEB10s-C_DEB1s)/V1 + k_2*exp(-E_2/R/T_1s)*(C_EB1s*C_E1s) - k_3*exp(-E_3/R/T_1s)*(C_B1s*C_DEB1s)
    dCdeb2_dt = F1*(C_DEB1s-C_DEB2s)/V2  + F2_new*(C_DEB20s-C_DEB2s)/V2        + k_2*exp(-E_2/R/T_2s)*(C_EB2s*C_E2s) - k_3*exp(-E_3/R/T_2s)*(C_B2s*C_DEB2s)
    dCe1_dt   = F1*(C_E10s-C_E1s)/V1     - k_1*exp(-E_1/R/T_1s)*(C_B1s*C_E1s)  - k_2*exp(-E_2/R/T_1s)*(C_EB1s*C_E1s)
    dCe2_dt   = F1*(C_E1s-C_E2s)/V2      + F2_new*(C_E20s-C_E2s)/V2            - k_1*exp(-E_1/R/T_2s)*(C_B2s*C_E2s)  - k_2*exp(-E_2/R/T_2s)*(C_EB2s*C_E2s)
    dCeb1_dt  = F1*(C_EB10s-C_EB1s)/V1   + k_1*exp(-E_1/R/T_1s)*(C_B1s*C_E1s)  - k_2*exp(-E_2/R/T_1s)*(C_EB1s*C_E1s) + 2*k_3*exp(-E_3/R/T_1s)*(C_B1s*C_DEB1s)
    dCeb2_dt  = F1*(C_EB1s-C_EB2s)/V2    + F2_new*(C_EB20s-C_EB2s)/V2          + k_1*exp(-E_1/R/T_2s)*(C_B2s*C_E2s)  - k_2*exp(-E_2/R/T_2s)*(C_EB2s*C_E2s) + 2*k_3*exp(-E_3/R/T_2s)*(C_B2s*C_DEB2s)
    dT1_dt    = F1*(T_10s-T_1s)/V1                         - H_1*k_1*exp(-E_1/R/T_1s)*(C_B1s*C_E1s)/(rhoe_L1*Cp1) - H_2*k_2*exp(-E_2/R/T_1s)*(C_EB1s*C_E1s)/(rhoe_L1*Cp1)- H_3*k_3*exp(-E_3/R/T_1s)*(C_B1s*C_DEB1s)/(rhoe_L1*Cp1) +Q1s/(rhoe_L1 * Cp1 * V1)
    dT2_dt    = F1*(T_1s-T_2s)/V2 + F2_new*(T_20s-T_2s)/V2 - H_1*k_1*exp(-E_1/R/T_2s)*(C_B2s*C_E2s)/(rhoe_L2*Cp2) - H_2*k_2*exp(-E_2/R/T_2s)*(C_EB2s*C_E2s)/(rhoe_L2*Cp2)- H_3*k_3*exp(-E_3/R/T_2s)*(C_B2s*C_DEB2s)/(rhoe_L2*Cp2) +Q2s/(rhoe_L2 * Cp2 * V2)
    return np.array([dCb1_dt,dCdeb1_dt,dCe1_dt,dCeb1_dt,dT1_dt,dCb2_dt,dCdeb2_dt,dCe2_dt,dCeb2_dt,dT2_dt], dtype=float)
timeFunction(seriesCSTR,[k_1,k_2,k_3,Cp1,Cp2], runs=100, per_run=1000)
timeFunction(seriesCSTRSS,[k_1,k_2,k_3,Cp1,Cp2], runs=100, per_run=1000)